In [0]:
# Configuración súper simple con Access Key
storage_account_name = "sistecreditofinal"
storage_account_access_key = ""
container_name = "raw"
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_access_key
)